<a href="https://colab.research.google.com/github/didulanthaisuru/time_series_forecasting/blob/main/nbeat_balance_final_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

sample_data_path = '/content/drive/MyDrive/Colab Notebooks'

In [4]:
files=os.listdir(sample_data_path)

In [5]:
!pip install pytorch_forecasting pytorch_lightning torch


In [6]:
print(files)

['Untitled2.ipynb', 'lstm.ipynb', 'cluster3_payments_only.csv', 'n_beats.ipynb', '.ipynb_checkpoints', 'n_beats_for_cluster threeipynb.ipynb', 'dataset_with_zeros.csv', 'colab_n_beats_modarate.ipynb', 'Untitled0.ipynb', 'nadil_category_expenses.xlsx', 'Untitled1.ipynb', 'n_beats_balance.xlsx', 'n_beats_balace_prediction.ipynb']


In [7]:
ff='/content/drive/MyDrive/Colab Notebooks/n_beats_balance.xlsx'

In [10]:
df=pd.read_excel(ff,engine='openpyxl')

In [9]:
import pandas as pd

In [11]:
print(df.head())
#print column names and datatypes
print(df.dtypes)

        Date  Normalized_Balance
0 2022-09-29            0.000539
1 2022-09-30            0.000539
2 2022-10-01            0.000539
3 2022-10-02            0.000539
4 2022-10-03            0.000539
Date                  datetime64[ns]
Normalized_Balance           float64
dtype: object


In [12]:
df["Date"]=pd.to_datetime(df["Date"])

In [13]:
df = df.sort_values("Date")
df = df.reset_index(drop=True)


In [14]:
!pip install neuralforecast


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.6 MB/s eta 0:00:00


In [15]:
from neuralforecast.core import NeuralForecast
from neuralforecast.models import NBEATS
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [16]:
# Rename columns
df_nf = df.rename(columns={'Date': 'ds', 'Normalized_Balance': 'y'})
df_nf['unique_id'] = 'balance'  # Only 1 time series, give it a name
df_nf = df_nf[['unique_id', 'ds', 'y']]

In [17]:
# Define model with horizon (future steps you want to predict)
horizon = 7  # for 7-day forecast

model = NeuralForecast(
    models=[NBEATS(input_size=30, h=horizon)],
    freq='D'  # daily data
)

# Fit the model
model.fit(df_nf)


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
555       Non-trainable params
2.4 M     Total params
9.786     Total estimated model params size (MB)
31        Modules in train mode
0       

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_steps=1000` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


In [18]:
future = model.predict()

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [24]:
print(future)
print(future.dtypes)

  unique_id         ds    NBEATS
0   balance 2025-02-01 -0.010337
1   balance 2025-02-02 -0.009911
2   balance 2025-02-03 -0.015047
3   balance 2025-02-04 -0.014537
4   balance 2025-02-05  0.002013
5   balance 2025-02-06  0.007067
6   balance 2025-02-07  0.011287
unique_id            object
ds           datetime64[ns]
NBEATS              float32
dtype: object


In [20]:
print(df.head())

        Date  Normalized_Balance
0 2022-09-29            0.000539
1 2022-09-30            0.000539
2 2022-10-01            0.000539
3 2022-10-02            0.000539
4 2022-10-03            0.000539


In [21]:
original_min=960.0
original_max=75138.99

In [25]:
future["Predicted_Balance"] = future["NBEATS"] * (original_max - original_min) + original_min

In [26]:
print(future)

  unique_id         ds    NBEATS  Predicted_Balance
0   balance 2025-02-01 -0.010337         193.217773
1   balance 2025-02-02 -0.009911         224.808350
2   balance 2025-02-03 -0.015047        -156.196167
3   balance 2025-02-04 -0.014537        -118.307251
4   balance 2025-02-05  0.002013        1109.313354
5   balance 2025-02-06  0.007067        1484.247070
6   balance 2025-02-07  0.011287        1797.228882
